##### Copyright 2018 The TensorFlow Probability Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License"); { display-mode: "form" }
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow 분포 형상의 이해

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/probability/examples/Understanding_TensorFlow_Distributions_Shapes"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a>   </td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ko/probability/examples/Understanding_TensorFlow_Distributions_Shapes.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ko/probability/examples/Understanding_TensorFlow_Distributions_Shapes.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ko/probability/examples/Understanding_TensorFlow_Distributions_Shapes.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
</table>

In [ ]:
import collections

import tensorflow as tf
tf.compat.v2.enable_v2_behavior()

import tensorflow_probability as tfp
tfd = tfp.distributions
tfb = tfp.bijectors

## 기초

TensorFlow 분포 형상과 관련된 세 가지 중요한 개념이 있습니다.

- *이벤트 형상*은 분포에서 단일 추출의 형상을 설명합니다. 추출은 차원에 따라 달라질 수 있습니다. 스칼라 분포의 경우 이벤트 형상은 `[]`입니다. 5차원 MultivariateNormal의 경우 이벤트 형상은 `[5]`입니다.
- *배치 형상*은 분포의 "배치"라고도 하는 동일하게 분포되지 않은 독립적인 추출을 설명합니다.
- *샘플 형상*은 분포군에서 독립적이고 동일하게 분포된 배치 추출을 설명합니다.

이벤트 형상과 배치 형상은 `Distribution` 객체의 속성인 반면, 샘플 형상은 `sample` 또는 `log_prob`에 대한 특정 호출과 연관됩니다.

이 노트북의 목적은 예제를 통해 이러한 개념을 설명하는 것이므로 즉시 이해가 되지 않더라도 걱정하지 마세요!

이러한 개념에 대한 또 다른 개념적 개요는 [이 블로그 게시물](https://ericmjl.github.io/blog/2019/5/29/reasoning-about-shapes-and-probability-distributions/)을 참조하세요.

### TensorFlow Eager에 대한 참고 사항

이 전체 노트북은 [TensorFlow Eager](https://research.googleblog.com/2017/10/eager-execution-imperative-define-by.html)를 사용하여 작성되었습니다. 제시된 어떤 개념에도 Eager가 *사용*되지는 않습니다. 다만 Eager의 경우, Python에서 `Distribution` 객체가 생성될 때 분포 배치와 이벤트 형상이 평가되지만(따라서 알려짐), 그래프(비 Eager 모드)에서는 그래프가 실행될 때까지 이벤트와 배치 형상이 결정되지 않는 분포를 정의할 수 있습니다.

## 스칼라 분포

위에서 언급했듯이 `Distribution` 객체는 이벤트와 배치 형상을 정의했습니다. 분포를 설명하는 유틸리티부터 시작하겠습니다.

In [ ]:
def describe_distributions(distributions):
  print('\n'.join([str(d) for d in distributions]))

이 섹션에서는 *스칼라* 분포, 즉 이벤트 형상이 `[]`인 분포를 살펴봅니다. `rate`에 의해 지정된 포아송 분포가 일반적인 예입니다.

In [5]:
poisson_distributions = [
    tfd.Poisson(rate=1., name='One Poisson Scalar Batch'),
    tfd.Poisson(rate=[1., 10., 100.], name='Three Poissons'),
    tfd.Poisson(rate=[[1., 10., 100.,], [2., 20., 200.]],
                name='Two-by-Three Poissons'),
    tfd.Poisson(rate=[1.], name='One Poisson Vector Batch'),
    tfd.Poisson(rate=[[1.]], name='One Poisson Expanded Batch')
]

describe_distributions(poisson_distributions)

tfp.distributions.Poisson("One_Poisson_Scalar_Batch", batch_shape=[], event_shape=[], dtype=float32)
tfp.distributions.Poisson("Three_Poissons", batch_shape=[3], event_shape=[], dtype=float32)
tfp.distributions.Poisson("Two_by_Three_Poissons", batch_shape=[2, 3], event_shape=[], dtype=float32)
tfp.distributions.Poisson("One_Poisson_Vector_Batch", batch_shape=[1], event_shape=[], dtype=float32)
tfp.distributions.Poisson("One_Poisson_Expanded_Batch", batch_shape=[1, 1], event_shape=[], dtype=float32)


포아송 분포는 스칼라 분포이므로 이벤트 형상은 항상 `[]`입니다. 더 많은 비율을 지정하면 배치 형상으로 나타납니다. 마지막 예제 쌍은 흥미롭습니다. 단 하나의 비율만 있지만 이 비율이 비어 있지 않은 형상의 numpy 배열에 포함되었기 때문에 이 형상은 배치 형상이 됩니다.

표준 정규 분포도 스칼라입니다. 이벤트 형상은 포아송의 경우와 같이 `[]`이지만 *브로드캐스팅*의 첫 번째 예를 보기 위해 사용해 보겠습니다. 이 정규 분포는 `loc` 및 `scale` 매개변수를 사용하여 지정됩니다.

In [6]:
normal_distributions = [
    tfd.Normal(loc=0., scale=1., name='Standard'),
    tfd.Normal(loc=[0.], scale=1., name='Standard Vector Batch'),
    tfd.Normal(loc=[0., 1., 2., 3.], scale=1., name='Different Locs'),
    tfd.Normal(loc=[0., 1., 2., 3.], scale=[[1.], [5.]],
               name='Broadcasting Scale')
]

describe_distributions(normal_distributions)

tfp.distributions.Normal("Standard", batch_shape=[], event_shape=[], dtype=float32)
tfp.distributions.Normal("Standard_Vector_Batch", batch_shape=[1], event_shape=[], dtype=float32)
tfp.distributions.Normal("Different_Locs", batch_shape=[4], event_shape=[], dtype=float32)
tfp.distributions.Normal("Broadcasting_Scale", batch_shape=[2, 4], event_shape=[], dtype=float32)


위의 흥미로운 예는 `Broadcasting Scale` 분포입니다. `loc` 매개변수의 형상은 `[4]`이고 `scale` 매개변수의 형상은 `[2, 1]`입니다. [Numpy 브로드캐스팅 규칙](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html)을 사용하면 배치 형상은 `[2, 4]`입니다. `"Broadcasting Scale"` 분포를 정의하는 동등한(그러나 덜 우아하고 권장되지 않는) 방법은 다음과 같습니다.

In [7]:
describe_distributions(
    [tfd.Normal(loc=[[0., 1., 2., 3], [0., 1., 2., 3.]],
                scale=[[1., 1., 1., 1.], [5., 5., 5., 5.]])])

tfp.distributions.Normal("Normal", batch_shape=[2, 4], event_shape=[], dtype=float32)


브로드캐스팅 표기가 두통을 일으키고 버그의 원인이 되기도 하지만 왜 유용한지 알 수 있습니다.

### 스칼라 분포 샘플링

분포로 할 수 있는 두 가지 주요 작업이 있습니다. 이로부터 `sample`할 수 있고 `log_prob`를 계산할 수 있습니다. 먼저 샘플링을 살펴보겠습니다. 기본 규칙은 분포에서 샘플링할 때 결과적인 Tensor의 형상은 `[sample_shape, batch_shape, event_shape]`이라는 것입니다. 여기서 `batch_shape` 및 `event_shape`는 `Distribution` 객체에 의해, `sample_shape`은 `sample`에 대한 호출에 의해 제공됩니다. 스칼라 분포의 경우 `event_shape = []`이므로 샘플에서 반환된 텐서의 형상은 `[sample_shape, batch_shape]`입니다. 시도해 보겠습니다.

In [8]:
def describe_sample_tensor_shape(sample_shape, distribution):
    print('Sample shape:', sample_shape)
    print('Returned sample tensor shape:',
          distribution.sample(sample_shape).shape)

def describe_sample_tensor_shapes(distributions, sample_shapes):
    started = False
    for distribution in distributions:
      print(distribution)
      for sample_shape in sample_shapes:
        describe_sample_tensor_shape(sample_shape, distribution)
      print()

sample_shapes = [1, 2, [1, 5], [3, 4, 5]]
describe_sample_tensor_shapes(poisson_distributions, sample_shapes)

tfp.distributions.Poisson("One_Poisson_Scalar_Batch", batch_shape=[], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1,)
Sample shape: 2
Returned sample tensor shape: (2,)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5)

tfp.distributions.Poisson("Three_Poissons", batch_shape=[3], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 3)
Sample shape: 2
Returned sample tensor shape: (2, 3)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 3)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 3)

tfp.distributions.Poisson("Two_by_Three_Poissons", batch_shape=[2, 3], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 2, 3)
Sample shape: 2
Returned sample tensor shape: (2, 2, 3)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 2, 3)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 2, 3)

In [9]:
describe_sample_tensor_shapes(normal_distributions, sample_shapes)

tfp.distributions.Normal("Standard", batch_shape=[], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1,)
Sample shape: 2
Returned sample tensor shape: (2,)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5)

tfp.distributions.Normal("Standard_Vector_Batch", batch_shape=[1], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 1)
Sample shape: 2
Returned sample tensor shape: (2, 1)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 1)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 1)

tfp.distributions.Normal("Different_Locs", batch_shape=[4], event_shape=[], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 4)
Sample shape: 2
Returned sample tensor shape: (2, 4)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 4)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 4)

tfp.distributions.Normal("Broadc

이것이 `sample`에 대해 말할 수 있는 전부입니다. 즉, 반환된 샘플 텐서는 `[sample_shape, batch_shape, event_shape]`의 형상을 갖습니다.

### 스칼라 분포에 대한 `log_prob` 계산

이제 좀 더 까다로운 `log_prob`을 살펴보겠습니다. `log_prob`는 분포에 대한 `log_prob`를 계산할 위치를 나타내는 (비어 있지 않은) 텐서를 입력으로 사용합니다. 가장 간단한 경우에 이 텐서는 `[sample_shape, batch_shape, event_shape]` 형식의 형상을 갖습니다. 여기서 `batch_shape` 및 `event_shape`는 분포의 배치 및 이벤트 형성과 일치합니다. 스칼라 분포의 경우 `event_shape = []`이므로 입력 텐서는 `[sample_shape, batch_shape]`의 형상을 갖는다는 점을 다시 한 번 상기하세요. 이 경우에 `[sample_shape, batch_shape]` 형상의 텐서를 다시 얻습니다.

In [10]:
three_poissons = tfd.Poisson(rate=[1., 10., 100.], name='Three Poissons')
three_poissons

<tfp.distributions.Poisson 'Three_Poissons' batch_shape=[3] event_shape=[] dtype=float32>

In [11]:
three_poissons.log_prob([[1., 10., 100.], [100., 10., 1]])  # sample_shape is [2].

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -2.0785608,   -3.2223587],
       [-364.73938  ,   -2.0785608,  -95.39484  ]], dtype=float32)>

In [12]:
three_poissons.log_prob([[[[1., 10., 100.], [100., 10., 1.]]]])  # sample_shape is [1, 1, 2].

<tf.Tensor: shape=(1, 1, 2, 3), dtype=float32, numpy=
array([[[[  -1.       ,   -2.0785608,   -3.2223587],
         [-364.73938  ,   -2.0785608,  -95.39484  ]]]], dtype=float32)>

첫 번째 예에서 입력과 출력의 형상이 `[2, 3]`이고 두 번째 예에서는 형상이 `[1, 1, 2, 3]`인 이유에 주목하세요.

브로드캐스팅이 아니었다면 할 말은 이 정도였을 것입니다. 브로드캐스팅을 고려했을 때의 규칙은 다음과 같습니다. 우리는 이것을 완전히 일반화하여 설명하고 스칼라 분포에 대한 단순화에 주목합니다.

1. `n = len(batch_shape) + len(event_shape)`를 정의합니다. (스칼라 분포의 경우 `len(event_shape)=0`입니다.)
2. 입력 텐서 `t`가 `n` 미만의 차원을 가지고 있는 경우, 정확히 `n` 차원이 될 때까지 왼쪽에 `1` 크기의 차원을 추가하여 형상을 채웁니다. 결과적인 텐서 `t'`를 호출합니다.
3. `log_prob`를 계산하는 대상 분포의 `[batch_shape, event_shape]`에 대해 `t'`의 가장 오른쪽 `n` 차원을 브로드캐스트합니다. 더 자세히 말하면, `t'`가 이미 분포와 일치하는 차원에 대해서는 아무것도 하지 않고 `t'`에 단일 항목이 있는 차원에 대해 해당 단일 항목을 적절한 횟수만큼 복제합니다. 다른 모든 상황은 오류입니다. (스칼라 분포의 경우, event_shape = `[]`이기 때문에 `batch_shape`에 대해서만 브로드캐스트합니다.)
4. 이제 마지막으로 `log_prob`를 계산할 수 있습니다. 결과 텐서는 `[sample_shape, batch_shape]` 형상을 가지며, 여기서 `sample_shape`는 `n` 맨 오른쪽 차원 `sample_shape = shape(t)[:-n]`의 왼쪽에 있는 `t` 또는 `t'` 차원으로 정의됩니다.

의미를 모르는 경우 혼란스러울 수 있으므로 몇 가지 예를 살펴보겠습니다.

In [13]:
three_poissons.log_prob([10.])

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([-16.104412 ,  -2.0785608, -69.05272  ], dtype=float32)>

텐서 `[10.]`(형상 `[1]`을 가짐)은 3의 `batch_shape` 전체에 걸쳐 브로드캐스트되므로 값 10에서 세 개의 포아송 로그 확률을 모두 평가합니다.

In [14]:
three_poissons.log_prob([[[1.], [10.]], [[100.], [1000.]]])

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[-1.0000000e+00, -7.6974149e+00, -9.5394836e+01],
        [-1.6104412e+01, -2.0785608e+00, -6.9052719e+01]],

       [[-3.6473938e+02, -1.4348087e+02, -3.2223587e+00],
        [-5.9131279e+03, -3.6195427e+03, -1.4069575e+03]]], dtype=float32)>

위의 예에서 입력 텐서는 형상이 `[2, 2, 1]`인 반면, 분포 객체는 배치 형상이 3입니다. 따라서 각 `[2, 2]` 샘플 차원에 대해 제공된 단일 값은 세 개의 포아송 각각에 브로드캐스트됩니다.

생각할 수 있는 유용한 방법: `three_poissons`에는 `batch_shape = [2, 3]`이 있기 때문에 `log_prob`에 대한 호출은 마지막 차원이 1 또는 3인 텐서를 취해야 합니다. 다른 모든 것은 오류입니다. (numpy 브로드캐스팅 규칙은 스칼라의 특수한 경우를 형상 `[1]`의 텐서와 완전히 동등한 것으로 취급합니다.)

`batch_shape = [2, 3]`을 이용한 더 복잡한 포아송 분포를 사용하여 절단을 테스트해 보겠습니다.

In [ ]:
poisson_2_by_3 = tfd.Poisson(
    rate=[[1., 10., 100.,], [2., 20., 200.]],
    name='Two-by-Three Poissons')

In [16]:
poisson_2_by_3.log_prob(1.)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [  -1.3068528,  -17.004269 , -194.70169  ]], dtype=float32)>

In [17]:
poisson_2_by_3.log_prob([1.])  # Exactly equivalent to above, demonstrating the scalar special case.

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [  -1.3068528,  -17.004269 , -194.70169  ]], dtype=float32)>

In [18]:
poisson_2_by_3.log_prob([[1., 1., 1.], [1., 1., 1.]])  # Another way to write the same thing. No broadcasting.

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [  -1.3068528,  -17.004269 , -194.70169  ]], dtype=float32)>

In [19]:
poisson_2_by_3.log_prob([[1., 10., 100.]])  # Input is [1, 3] broadcast to [2, 3].

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ -1.       ,  -2.0785608,  -3.2223587],
       [ -1.3068528,  -5.14709  , -33.90767  ]], dtype=float32)>

In [20]:
poisson_2_by_3.log_prob([[1., 10., 100.], [1., 10., 100.]])  # Equivalent to above. No broadcasting.

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ -1.       ,  -2.0785608,  -3.2223587],
       [ -1.3068528,  -5.14709  , -33.90767  ]], dtype=float32)>

In [21]:
poisson_2_by_3.log_prob([[1., 1., 1.], [2., 2., 2.]])  # No broadcasting.

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [  -1.3068528,  -14.701683 , -190.09653  ]], dtype=float32)>

In [22]:
poisson_2_by_3.log_prob([[1.], [2.]])  # Equivalent to above. Input shape [2, 1] broadcast to [2, 3].

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [  -1.3068528,  -14.701683 , -190.09653  ]], dtype=float32)>

위의 예에는 배치에 대한 브로드캐스팅이 포함되어 있지만 샘플 형상은 비어 있습니다. 값 모음이 있고 배치의 각 지점에서 각 값의 로그 확률을 얻으려고 한다고 가정합니다. 수동으로 할 수 있습니다.

In [23]:
poisson_2_by_3.log_prob([[[1., 1., 1.], [1., 1., 1.]], [[2., 2., 2.], [2., 2., 2.]]])  # Input shape [2, 2, 3].

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39484  ],
        [  -1.3068528,  -17.004269 , -194.70169  ]],

       [[  -1.6931472,   -6.087977 ,  -91.48282  ],
        [  -1.3068528,  -14.701683 , -190.09653  ]]], dtype=float32)>

또는 브로드캐스팅이 마지막 배치 차원을 처리하도록 할 수 있습니다.

In [24]:
poisson_2_by_3.log_prob([[[1.], [1.]], [[2.], [2.]]])  # Input shape [2, 2, 1].

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39484  ],
        [  -1.3068528,  -17.004269 , -194.70169  ]],

       [[  -1.6931472,   -6.087977 ,  -91.48282  ],
        [  -1.3068528,  -14.701683 , -190.09653  ]]], dtype=float32)>

또한 브로드캐스팅이 첫 번째 배치 차원만 처리하도록 할 수도 있습니다(아마도 다소 덜 자연스러운 방법임).

In [25]:
poisson_2_by_3.log_prob([[[1., 1., 1.]], [[2., 2., 2.]]])  # Input shape [2, 1, 3].

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39484  ],
        [  -1.3068528,  -17.004269 , -194.70169  ]],

       [[  -1.6931472,   -6.087977 ,  -91.48282  ],
        [  -1.3068528,  -14.701683 , -190.09653  ]]], dtype=float32)>

또는 브로드캐스팅이 *두* 배치 차원을 모두 처리하도록 할 수 있습니다.

In [26]:
poisson_2_by_3.log_prob([[[1.]], [[2.]]])  # Input shape [2, 1, 1].

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39484  ],
        [  -1.3068528,  -17.004269 , -194.70169  ]],

       [[  -1.6931472,   -6.087977 ,  -91.48282  ],
        [  -1.3068528,  -14.701683 , -190.09653  ]]], dtype=float32)>

위의 방법은 원하는 값이 두 개 뿐일 때 제대로 작동했지만 모든 배치 지점에서 평가하려는 값이 많다고 가정해 보겠습니다. 이를 위해 형상 오른쪽에 크기 1의 추가 차원을 추가하는 다음 표기법이 매우 유용합니다.

In [27]:
poisson_2_by_3.log_prob(tf.constant([1., 2.])[..., tf.newaxis, tf.newaxis])

<tf.Tensor: shape=(2, 2, 3), dtype=float32, numpy=
array([[[  -1.       ,   -7.697415 ,  -95.39484  ],
        [  -1.3068528,  -17.004269 , -194.70169  ]],

       [[  -1.6931472,   -6.087977 ,  -91.48282  ],
        [  -1.3068528,  -14.701683 , -190.09653  ]]], dtype=float32)>

이것은 [스트라이드 슬라이스 표기법](https://www.tensorflow.org/api_docs/cc/class/tensorflow/ops/strided-slice)의 예로, 알아두면 도움이 됩니다.

완전하게 하기 위해 `three_poissons`로 돌아가면 동일한 예는 다음과 같은 모양입니다.

In [28]:
three_poissons.log_prob([[1.], [10.], [50.], [100.]])

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [ -16.104412 ,   -2.0785608,  -69.05272  ],
       [-149.47777  ,  -43.34851  ,  -18.219261 ],
       [-364.73938  , -143.48087  ,   -3.2223587]], dtype=float32)>

In [29]:
three_poissons.log_prob(tf.constant([1., 10., 50., 100.])[..., tf.newaxis])  # Equivalent to above.

<tf.Tensor: shape=(4, 3), dtype=float32, numpy=
array([[  -1.       ,   -7.697415 ,  -95.39484  ],
       [ -16.104412 ,   -2.0785608,  -69.05272  ],
       [-149.47777  ,  -43.34851  ,  -18.219261 ],
       [-364.73938  , -143.48087  ,   -3.2223587]], dtype=float32)>

## 다변량 분포

이제 비어 있지 않은 이벤트 형상을 갖는 다변량 분포로 전환합니다. 다항 분포를 살펴보겠습니다.

In [30]:
multinomial_distributions = [
    # Multinomial is a vector-valued distribution: if we have k classes,
    # an individual sample from the distribution has k values in it, so the
    # event_shape is `[k]`.
    tfd.Multinomial(total_count=100., probs=[.5, .4, .1],
                    name='One Multinomial'),
    tfd.Multinomial(total_count=[100., 1000.], probs=[.5, .4, .1],
                    name='Two Multinomials Same Probs'),
    tfd.Multinomial(total_count=100., probs=[[.5, .4, .1], [.1, .2, .7]],
                    name='Two Multinomials Same Counts'),
    tfd.Multinomial(total_count=[100., 1000.],
                    probs=[[.5, .4, .1], [.1, .2, .7]],
                    name='Two Multinomials Different Everything')

]

describe_distributions(multinomial_distributions)

tfp.distributions.Multinomial("One_Multinomial", batch_shape=[], event_shape=[3], dtype=float32)
tfp.distributions.Multinomial("Two_Multinomials_Same_Probs", batch_shape=[2], event_shape=[3], dtype=float32)
tfp.distributions.Multinomial("Two_Multinomials_Same_Counts", batch_shape=[2], event_shape=[3], dtype=float32)
tfp.distributions.Multinomial("Two_Multinomials_Different_Everything", batch_shape=[2], event_shape=[3], dtype=float32)


마지막 세 가지 예에서 batch_shape이 항상 `[2]`이지만 브로드캐스팅을 사용하여 공유 `total_count` 또는 공유 `probs`를 가질 수 있는(또는 둘 다 가질 수 없는) 이유에 주목하세요. 왜냐하면 막후에서는 동일한 형상을 갖도록 브로드캐스트되기 때문입니다.

우리가 이미 알고 있는 것을 감안할 때 샘플링은 간단합니다.

In [31]:
describe_sample_tensor_shapes(multinomial_distributions, sample_shapes)

tfp.distributions.Multinomial("One_Multinomial", batch_shape=[], event_shape=[3], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 3)
Sample shape: 2
Returned sample tensor shape: (2, 3)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 3)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 3)

tfp.distributions.Multinomial("Two_Multinomials_Same_Probs", batch_shape=[2], event_shape=[3], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 2, 3)
Sample shape: 2
Returned sample tensor shape: (2, 2, 3)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 2, 3)
Sample shape: [3, 4, 5]
Returned sample tensor shape: (3, 4, 5, 2, 3)

tfp.distributions.Multinomial("Two_Multinomials_Same_Counts", batch_shape=[2], event_shape=[3], dtype=float32)
Sample shape: 1
Returned sample tensor shape: (1, 2, 3)
Sample shape: 2
Returned sample tensor shape: (2, 2, 3)
Sample shape: [1, 5]
Returned sample tensor shape: (1, 5, 2, 3)
Sample shape: [3, 4, 5]


로그 확률을 계산하는 것도 마찬가지로 간단합니다. 대각선 다변량 정규 분포의 예를 살펴보겠습니다. (카운트와 확률에 대한 제약으로 인해 브로드캐스팅으로 인해 종종 허용 불가한 값이 생성되기 때문에 다항식은 브로드캐스트에 그리 친화적이지 않습니다.) 평균은 같지만 스케일(표준 편차)이 다른 2개의 3차원 분포 배치를 사용할 것입니다.

In [32]:
two_multivariate_normals = tfd.MultivariateNormalDiag(loc=[1., 2., 3.], scale_identity_multiplier=[1., 2.])
two_multivariate_normals

<tfp.distributions.MultivariateNormalDiag 'MultivariateNormalDiag' batch_shape=[2] event_shape=[3] dtype=float32>

(스케일이 ID의 배수인 분포를 사용했지만 이것은 제한 사항이 아니라는 점에 주목하세요. `scale_identity_multiplier` 대신 `scale`을 전달할 수 있습니다.)

이제 평균과 이동된 평균에서 각 배치 지점의 로그 확률을 평가해 보겠습니다.

In [33]:
two_multivariate_normals.log_prob([[[1., 2., 3.]], [[3., 4., 5.]]])  # Input has shape [2,1,3].

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-2.7568154, -4.836257 ],
       [-8.756816 , -6.336257 ]], dtype=float32)>

정확히 동일하게, [https://www.tensorflow.org/api_docs/cc/class/tensorflow/ops/strided-slice](스트라이드 슬라이드 표기)를 사용하여 상수 중간에 추가 형상=1 차원을 삽입할 수 있습니다.

In [34]:
two_multivariate_normals.log_prob(
    tf.constant([[1., 2., 3.], [3., 4., 5.]])[:, tf.newaxis, :])  # Equivalent to above.

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[-2.7568154, -4.836257 ],
       [-8.756816 , -6.336257 ]], dtype=float32)>

한편, 추가 차원을 삽입하지 않으면 `[1., 2., 3.]`을 첫 번째 배치 지점으로 전달하고 `[3., 4., 5.]`를 두 번째 배치 지점으로 전달합니다.

In [35]:
two_multivariate_normals.log_prob(tf.constant([[1., 2., 3.], [3., 4., 5.]]))

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-2.7568154, -6.336257 ], dtype=float32)>

## 형상 조작 기법

### Reshape 바이젝터

`Reshape` 바이젝터는 분포의 *event_shape* 형상을 변경하는 데 사용할 수 있습니다. 예를 들어 보겠습니다.

In [36]:
six_way_multinomial = tfd.Multinomial(total_count=1000., probs=[.3, .25, .2, .15, .08, .02])
six_way_multinomial

<tfp.distributions.Multinomial 'Multinomial' batch_shape=[] event_shape=[6] dtype=float32>

이벤트 형상이 `[6]`인 다항식을 만들었습니다. Reshape 바이젝터를 사용하면 이것을 이벤트 형상이 `[2, 3]`인 분포로 취급할 수 있습니다.

`Bijector`는 ${\mathbb R}^n$의 개방된 부분 집합에서 미분 가능한 일대일 함수를 나타냅니다. `Bijectors`는 기본 분포 $p(x)$ 및 $Y = g(X)$를 나타내는 `Bijector` 측면에서 분포 $p(y)$를 모델링하는 `TransformedDistribution`과 함께 사용됩니다. 실제로 살펴보겠습니다.

In [37]:
transformed_multinomial = tfd.TransformedDistribution(
    distribution=six_way_multinomial,
    bijector=tfb.Reshape(event_shape_out=[2, 3]))
transformed_multinomial

<tfp.distributions.TransformedDistribution 'reshapeMultinomial' batch_shape=[] event_shape=[2, 3] dtype=float32>

In [38]:
six_way_multinomial.log_prob([500., 100., 100., 150., 100., 50.])

<tf.Tensor: shape=(), dtype=float32, numpy=-178.22021>

In [39]:
transformed_multinomial.log_prob([[500., 100., 100.], [150., 100., 50.]])

<tf.Tensor: shape=(), dtype=float32, numpy=-178.22021>

이것은 `Reshape` 바이젝터가 할 수 있는 *유일한* 일입니다. 이벤트 차원을 배치 차원으로, 또는 그 반대로 전환할 수 없습니다.

### 독립 분포

`Independent` 분포는 독립적이고 반드시 동일하지는 않은(일명 배치) 분포 모음을 단일 분포로 처리하는 데 사용됩니다. 더 간결하게 말하면, `Independent`는 `batch_shape`의 차원을 `event_shape`의 차원으로 전환할 수 있게 해줍니다. 예를 들면 다음과 같습니다.

In [40]:
two_by_five_bernoulli = tfd.Bernoulli(
    probs=[[.05, .1, .15, .2, .25], [.3, .35, .4, .45, .5]],
    name="Two By Five Bernoulli")
two_by_five_bernoulli

<tfp.distributions.Bernoulli 'Two_By_Five_Bernoulli' batch_shape=[2, 5] event_shape=[] dtype=int32>

우리는 이것을 앞면 확률이 관련된 2x5 동전 배열로 생각할 수 있습니다. 임의의 1과 0 집합의 확률을 평가해 보겠습니다.

In [41]:
pattern = [[1., 0., 0., 1., 0.], [0., 0., 1., 1., 1.]]
two_by_five_bernoulli.log_prob(pattern)

<tf.Tensor: shape=(2, 5), dtype=float32, numpy=
array([[-2.9957323 , -0.10536052, -0.16251892, -1.609438  , -0.2876821 ],
       [-0.35667497, -0.4307829 , -0.9162907 , -0.7985077 , -0.6931472 ]],
      dtype=float32)>

`Independent`를 사용하여 이것을 두 개의 서로 다른 "베르누이 5개 세트"로 바꿀 수 있습니다. 이는 주어진 패턴으로 나오는 동전 뒤집기 "행"을 단일 결과로 간주하려는 경우에 유용합니다.

In [42]:
two_sets_of_five = tfd.Independent(
    distribution=two_by_five_bernoulli,
    reinterpreted_batch_ndims=1,
    name="Two Sets Of Five")
two_sets_of_five

<tfp.distributions.Independent 'Two_Sets_Of_Five' batch_shape=[2] event_shape=[5] dtype=int32>

수학적으로, 집합에서 5번의 "독립적인" 동전 뒤집기의 로그 확률을 합산하여 5개 "세트" 각각의 로그 확률을 계산합니다. 이것이 이 분포의 이름이 생겨난 이유입니다.

In [43]:
two_sets_of_five.log_prob(pattern)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([-5.160732 , -3.1954036], dtype=float32)>

여기서 더 나아가 `Independent`를 사용하여 개별 이벤트가 2x5 베르누이 집합인 분포를 만들 수 있습니다.

In [44]:
one_set_of_two_by_five = tfd.Independent(
    distribution=two_by_five_bernoulli, reinterpreted_batch_ndims=2,
    name="One Set Of Two By Five")
one_set_of_two_by_five.log_prob(pattern)

<tf.Tensor: shape=(), dtype=float32, numpy=-8.356134>

`sample`의 관점에서 `Independent`를 사용해도 변하는 것은 아무것도 없다는 점에 주목할 가치가 있습니다.

In [45]:
describe_sample_tensor_shapes(
    [two_by_five_bernoulli,
     two_sets_of_five,
     one_set_of_two_by_five],
    [[3, 5]])

tfp.distributions.Bernoulli("Two_By_Five_Bernoulli", batch_shape=[2, 5], event_shape=[], dtype=int32)
Sample shape: [3, 5]
Returned sample tensor shape: (3, 5, 2, 5)

tfp.distributions.Independent("Two_Sets_Of_Five", batch_shape=[2], event_shape=[5], dtype=int32)
Sample shape: [3, 5]
Returned sample tensor shape: (3, 5, 2, 5)

tfp.distributions.Independent("One_Set_Of_Two_By_Five", batch_shape=[], event_shape=[2, 5], dtype=int32)
Sample shape: [3, 5]
Returned sample tensor shape: (3, 5, 2, 5)



독자를 위한 작별 연습으로 샘플링 및 로그 확률 관점에서 `Normal` 분포와 `MultivariateNormalDiag` 분포의 벡터 배치간 차이점과 유사점을 고려해볼 것을 제시합니다. `Independent`를 사용하여 `Normal` 배치로부터 `MultivariateNormalDiag`를 구성할 수 있는 방법은 무엇일까요? (단, `MultivariateNormalDiag`는 실제로 이런 식으로 구현되지는 않습니다.)